# CopasiProcess Composer

In [1]:
import sys 

sys.path.insert(0, '..')

In [2]:
import os 
from basico import *
from process_bigraph import Composite, pf 
from builder import Builder, Process as builderProcess, ProcessTypes
from biosimulator_processes.processes.copasi_process import CopasiProcess
from biosimulator_processes.process_types import MODEL_TYPE, CHANGES_SCHEMA
from biosimulator_processes.utils import play_composition

CobraProcess registered successfully.


TypeError: Too many parameters for typing.Dict; actual 3, expected 2

### Example 1: Define a single process instance and a corresponding emitter

##### 1. Define the schema by which our Composite instance will be configured. Here a user chooses an entrypoint through which to instantiate the composite process. We will use an SBML-encoded model file as our entrypoint in this example.


##### **As per the `CopasiProcess` constructor, there are three primary methods of entry:**
    

    # Parse values from the process `config` attribute dictionary:
    
        model_file = self.config.get('model').get('model_source')
        sed_model_id = self.config.get('model').get('model_id')
        biomodel_id = self.config.get('biomodel_id')
        source_model_id = biomodel_id or sed_model_id


    # A. enter with model_file
    
        if model_file:
            self.copasi_model_object = load_model(model_file)
    

    # B. enter with specific search term for a model
     
        elif source_model_id:
            self.copasi_model_object = fetch_biomodel(model_id=source_model_id)
    

    # C. enter with a new model
    
        else:
            self.copasi_model_object = new_model(name='CopasiProcess Model')

##### Here we define the schema for a single process based on the provided entrypoint.

In [ ]:
root = os.path.abspath('..')
model_path = os.path.join(root, 'biosimulator_processes/model_files/BIOMD0000000061_url.xml')

instance1 = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_source': model_path
            }
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    }
}

##### 2. Instantiate the Process-Bigraph-Engine with the Composite class

In [ ]:
workflow = Composite({
    'state': instance1
})

##### 3. Run the Composite workflow with the Engine

In [ ]:
workflow.run(10)

##### 4. Gather and neatly-print the results.

In [ ]:
results = workflow.gather_results()

print(f'RESULTS: {pf(results)}')

##### 5. Plot/Analyze the results with Plotly

In [ ]:
# 5. Plot/Analyze the results

#### Example 2: Define a single-process composition from a given BioModels model id and make model changes. The options available for configuration of a `'model'` that can be run by the composite process are based on SEDML L1V4 specifications. Please note that not all parameters are required in this configuration.

In [ ]:
CHANGES_SCHEMA

#### Example 2a: Define a single process with a different entrypoint.

#### Example 3: Define multiple processes in the same instance for a parameter scan. 

In [ ]:
from biosimulator_processes.utils import generate_copasi_parameter_scan_instance as param_scan

parameter_scan_instance = param_scan(num_iterations=100, entry_config={'biomodel_id': 'BIOMD0000000051'})